In [1]:
import sys
sys.path.append('..')
from anonymouus import Anonymize
from dynamic_substitution import DynamicSubstitution
from date_replacer import DateReplacer

In [2]:
d = DateReplacer()
d.clear_patterns()
from random import shuffle
def shuffle_date(word):
    word = list(word)
    shuffle(word)
    return ''.join(word)

repl = shuffle_date
#'1970-01-01'
d.add_pattern(r'(\d){4}[-/]{1}(\d){2}[-/]{1}(\d){2}', repl)

<class 'function'>


In [3]:
d.replace('het was op 1969-03-01!')

'het was op -6-0910193!'

In [4]:
d.clear_patterns()
d.add_pattern(r'(\d){4}[-/]{1}(\d){2}[-/]{1}(\d){2}', lambda x: x[::-1])

<class 'function'>


In [5]:
d.replace('het was op 1969-03-01!')

'het was op 10-30-9691!'